In [1]:
import gym
import collections

from torch.utils.tensorboard import SummaryWriter

In [2]:
ALPHA = 0.2
GAMMA = 0.9
NUM_TEST_EPISODES = 20
GYM_ENV = "FrozenLake-v0"

In [3]:
class Agent:
    def __init__(self):
        self.env = gym.make(GYM_ENV)
        self.state = self.env.reset()
        self.values = collections.defaultdict(float)
        
    def play_episode(self, env):
        """
        use built knowledge on outside env
        """
        total_reward = 0.0
        state = env.reset()
        while True:
            _, action = self.best_value_and_action(state)
            new_state, reward, is_done, _ = env.step(action)
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward
        
    def value_update(self, state, action, reward, next_state):
        """
        updates (old) state entry of value table given findings from action -> new state
        implements Bellman approx with learning rate
        """
        best_value, _ = self.best_value_and_action(next_state)
        new_value = reward + GAMMA * best_value
        old_value = self.values[(state, action)]
        self.values[(state, action)] = old_value * (1-ALPHA) + ALPHA * new_value
    
    def sample_env(self):
        """
        used for training to build knowledge of env
        """
        action = self.env.action_space.sample()
        old_state = self.state
        new_state, reward, is_done, _ = self.env.step(action)
        self.state = self.env.reset() if is_done else new_state
        return old_state, action, reward, new_state
    
    def best_value_and_action(self, state):
        """
        determines 'best' value and action at a state by argmaxing all saved state-action values
        """
        best_value, best_action = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or action_value > best_value:
                best_value = action_value
                best_action = action
        return best_value, best_action

In [4]:
test_env = gym.make(GYM_ENV)
agent = Agent()
writer = SummaryWriter(comment=f'-q-learning-{GYM_ENV}')

i = 0
best_reward = 0.0
while True:
    i += 1
    state, action, reward, next_state = agent.sample_env()
    agent.value_update(state, action, reward, next_state)
    
    reward = 0.0
    for _ in range(NUM_TEST_EPISODES):
        reward += agent.play_episode(test_env)
    reward /= NUM_TEST_EPISODES
    writer.add_scalar("reward", reward, i)
    
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
        best_reward = reward
    
    if reward > 0.8:
        print("Solved in %d iterations" % i)
        break

writer.close()

Best reward updated 0.000 -> 0.050
Best reward updated 0.050 -> 0.100
Best reward updated 0.100 -> 0.150
Best reward updated 0.150 -> 0.250
Best reward updated 0.250 -> 0.400
Best reward updated 0.400 -> 0.450
Best reward updated 0.450 -> 0.500
Best reward updated 0.500 -> 0.550
Best reward updated 0.550 -> 0.650
Best reward updated 0.650 -> 0.750
Best reward updated 0.750 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 11939 iterations
